Still to do
* Finish going through each column
* Copy code from datA_exploration notebook
* delete data_exploration notebook
* update the numbering on the notebooks
* carry on with next notebook

useful link (medium article)[https://towardsdatascience.com/an-extensive-guide-to-exploratory-data-analysis-ddd99a03199e]
useful kaggle [https://www.kaggle.com/hellbuoy/online-retail-k-means-hierarchical-clustering]

# Import Libraries

In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Set Seaborn Parameters

In [15]:
plt.rcParams["figure.figsize"] = (10, 10)

sns.set_style = "seaborn-whitegrid"

sns.set(
    rc={
        "font.style": "normal",
        "axes.facecolor": "white",
        "grid.color": ".8",
        "grid.linestyle": "-",
        "figure.facecolor": "white",
        "figure.titlesize": 20,
        "text.color": "black",
        "xtick.color": "black",
        "ytick.color": "black",
        "axes.labelcolor": "black",
        "axes.grid": True,
        "axes.labelsize": 10,
        "xtick.labelsize": 10,
        "font.size": 10,
        "ytick.labelsize": 10,
    }
)

# Import Data

In [116]:
df = pd.read_excel("../data/Online Retail.xlsx",
                   engine='openpyxl')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


# Data Overview

In [117]:
# Display the number of rows and columns for the dataset
df.shape

(541909, 8)

In [118]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [119]:
# Show the number of unique values in each column
df.nunique(axis=0)

InvoiceNo      25900
StockCode       4070
Description     4223
Quantity         722
InvoiceDate    23260
UnitPrice       1630
CustomerID      4372
Country           38
dtype: int64

In [120]:
# Summary of numeric variables
df.describe().apply(lambda s: s.apply(lambda x: format(x, 'f')))

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


* Negative values in "Quantity" seems odd, and will require further exploration
* The max values in "Quantity" and "UnitPrice" also seem odd when compared to the mean and standard deviation

## Null Values

In [121]:
# Show the percentage of missing values for each column
round(df.isnull().mean() * 100, 2)

InvoiceNo       0.00
StockCode       0.00
Description     0.27
Quantity        0.00
InvoiceDate     0.00
UnitPrice       0.00
CustomerID     24.93
Country         0.00
dtype: float64

* Only "Description" and "CustomerID" have missing values.
* In rows with no "Description", we still have other useful information, such as "StockCode" and "UnitPrice".  We should keep the rows which are only missing "Description".
* Our aim is to segment Customers.  If we don't know which customer made a particular purchase we no longer require the rest of the row's data.  We can therefore drop the rows without a "CustomerID".

In [124]:
## Drop rows with no "CustomerID"
rows_before = df.shape[0]
df.dropna(subset=['CustomerID'], inplace=True)
rows_after = df.shape[0]
print('Dropped {} rows with no value in "CustomerID"'.format(rows_before-rows_after))

Dropped 0 rows with no value in "CustomerID"


## Change Data Types

* The values in some columns merely represent unique values, and do not indicate a relationship to other values.  e.g. a CustomerID of 1000 is not twice the value of a CustomerID of 500.  The opposite would be something like UnitPrice, where 1000 is indeed twice the value of 500.  
* These unique ID columns should contain data as a string type.

In [151]:
df.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [152]:
# "InvoiceNo"
df['InvoiceNo'] = df['InvoiceNo'].astype(str)

# "StockCode"
df['StockCode'] = df['StockCode'].astype(str)

# "CustomerID"
df['CustomerID'] = df['CustomerID'].astype(str)

# Exploration of columns

## ID Columns

### InvoiceNo

* If an invoice is a refund, the InvoiceNo will begin with C

In [203]:
df[df['InvoiceNo'].str.match('C')].head(2)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527.0,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311.0,United Kingdom


* We are not concerned with refunds or discounts so should drop these rows

In [208]:
# Drop rows with InvoiceNo beggining with "C"
df = df[~df['InvoiceNo'].str.match('C')]

### StockCode

* StockCode with value of M = Manual, which still leaves valuable information, so no rows will be dropped.

In [210]:
df[df['StockCode'].str.len() < 2].head(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
2239,536569,M,Manual,1,2010-12-01 15:35:00,1.25,16274.0,United Kingdom
2250,536569,M,Manual,1,2010-12-01 15:35:00,18.95,16274.0,United Kingdom
6798,536981,M,Manual,2,2010-12-03 14:26:00,0.85,14723.0,United Kingdom


## Description

Check for the rows where its not all uppercase

In [217]:
# Top 10 most popular items
popular_items = df['Description'].value_counts()[:10]

In [219]:
popular_items

WHITE HANGING HEART T-LIGHT HOLDER    2028
REGENCY CAKESTAND 3 TIER              1724
JUMBO BAG RED RETROSPOT               1618
ASSORTED COLOUR BIRD ORNAMENT         1408
PARTY BUNTING                         1397
LUNCH BAG RED RETROSPOT               1316
SET OF 3 CAKE TINS PANTRY DESIGN      1159
LUNCH BAG  BLACK SKULL.               1105
POSTAGE                               1099
PACK OF 72 RETROSPOT CAKE CASES       1068
Name: Description, dtype: int64

In [220]:
# Plot of 10 most popular items
sns.barplot(popular_items.index, popular_items)

/Users/adam/opt/anaconda3/lib/python3.7/site-packages/seaborn/_decorators.py:43: FutureWarning: Pass the following variable as a keyword arg: x. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning


TypeError: Horizontal orientation requires numeric `x` variable.

## Quantity

## InvoiceDate

## UnitPrice

## CustomerID

In [125]:
df['CustomerID'].unique()

array(['17850.0', '13047.0', '12583.0', ..., '13298.0', '14569.0',
       '12713.0'], dtype=object)

## Country

# Add Useful Columns

In [ ]:
# Add a TotalPrice column based on Quantity multiplied by UnitPrice
# df['TotalPrice'] = df.loc[:,'Quantity'] * df.loc[:,'UnitPrice']

# Save Processed DataFrame as a CSV

In [ ]:
# Save as csv file, which is more compatible with pandas
df.to_csv('../data/online_retail_processed')